In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
teams_data = pd.read_csv("2023-07-26_weaver3_teams_bench_np1.csv")

In [ ]:
teams_data.head()

In [ ]:
data_by_mode = {}

In [ ]:
modenames = ["nview", "rview", "uview"]
modenums = [0,1,2]
team_sizes = [4,8,16,32,64,128,256,512,1024]

In [ ]:
# slice into dicts by league size
for (modename, modenum) in zip(modenames, modenums):
    data_by_mode[modename] = teams_data.loc[teams_data["mode"] == modenum]

In [ ]:
data_by_mode_ts = {}
for modename in modenames:
    data_by_mode_ts[modename] = {}
    for team_size in team_sizes:
        data_by_mode_ts[modename][team_size] = data_by_mode[modename].loc[data_by_mode[modename]["team_size"] == team_size]

In [ ]:
data_fixed_ts_nview = [data_by_mode_ts["nview"][ts] for ts in team_sizes]
data_fixed_ts_rview = [data_by_mode_ts["rview"][ts] for ts in team_sizes]
data_fixed_ts_uview = [data_by_mode_ts["uview"][ts] for ts in team_sizes]
labels_fixed_ts = ["team size "+str(ts) for ts in team_sizes]

In [ ]:
def figure_for_attribute(dfs, labels, attribute, xaxis="n", xlabel="problem size (array of size n)", ylabel = None, title = None, style="--", scale="linear"):
    if ylabel is None:
        ylabel = attribute
    
    if title is None:
        title = f"update benchmark ({attribute.replace('_',' ')})"
    
    if len(dfs) == 0:
        print("required to have at least one line to plot")
    
    maxy = max(dfs[0][f"{attribute}-mean"])
    miny = min(dfs[0][f"{attribute}-mean"])
    for df in dfs:
        cur_max = max(df[f"{attribute}-mean"])
        cur_min = min(df[f"{attribute}-mean"])
        if cur_max > maxy:
            maxy = cur_max
        if cur_min < miny:
            miny = cur_min
    
    if labels is None:
        print("required to label each line")
        
    plt.figure(figsize=[10,6])
    for df, dflabel in zip(dfs,labels):
        plt.plot(df[xaxis], df[f"{attribute}-mean"], style, label=dflabel)
    plt.title(title)
    plt.legend()
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xscale(scale)
    plt.yscale(scale)
    if scale == "linear":
        ylim_base = 0
    else:
        ylim_base = miny*0.95
    plt.ylim(ylim_base,int((maxy+1)*1.05))
    plt.grid()
    plt.savefig(f"{title}.png")
    plt.show()

In [ ]:
figure_for_attribute(data_fixed_ts_nview, labels_fixed_ts, "gups", xaxis="league_size", ylabel = "GStreamUpdates/second", xlabel = "league size", title="1-node normal view overhead time", style="*-", scale="log")

In [ ]:
figure_for_attribute(data_fixed_ts_rview, labels_fixed_ts, "gups", xaxis="league_size", ylabel = "GStreamUpdates/second", xlabel = "league size", title="1-node remote view overhead time", style="*-", scale="log")

In [ ]:
figure_for_attribute(data_fixed_ts_uview, labels_fixed_ts, "gups", xaxis="league_size", ylabel = "GStreamUpdates/second", xlabel = "league size", title="1-node unamaged view overhead time", style="*-", scale="log")